In [2]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
from keras import layers


In [3]:
def load_devanagari_dataset(dataset_path):
    images = []
    labels = []
    label_dict = {"consonants": 0, "vowels": 1, "digits": 2}

    for folder in os.listdir(dataset_path):
        label = label_dict.get(folder)
        if label is not None:
            folder_path = os.path.join(dataset_path, folder)
            for filename in os.listdir(folder_path):
                img_path = os.path.join(folder_path, filename)
                if os.path.isfile(img_path):
                    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                    if img is not None:
                        img = cv2.resize(img, (64, 64))  # Adjust the size as needed
                        img = img / 255.0  # Normalize pixel values to [0, 1]
                        images.append(img)
                        labels.append(label)

    return np.array(images), np.array(labels)


In [4]:
import os
import cv2
import numpy as np

def load_custom_dataset(dataset_path):
    images = []
    labels = []
    label_dict = {}  # Mapping from class name to label

    for label, class_name in enumerate(os.listdir(dataset_path)):
        label_dict[class_name] = label
        class_path = os.path.join(dataset_path, class_name)

        for filename in os.listdir(class_path):
            img_path = os.path.join(class_path, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

            if img is not None:
                img = cv2.resize(img, (64, 64))  # Adjust the size as needed
                img = img / 255.0  # Normalize pixel values to [0, 1]
                images.append(img)
                labels.append(label)

    # Convert lists to numpy arrays
    images = np.array(images)
    labels = np.array(labels)

    return images, labels, label_dict

# Example usage
dataset_path = "Downloads/images"
X, y, label_dict = load_custom_dataset(dataset_path)


In [6]:
# Assuming you have already loaded the dataset using the provided function
# X, y = load_devanagari_dataset(dataset_path)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert labels to one-hot encoding
num_classes = len(np.unique(y))
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Reshape the data for compatibility with Conv2D layers
X_train = X_train.reshape(X_train.shape[0], 64, 64, 1)
X_test = X_test.reshape(X_test.shape[0], 64, 64, 1)


# Print the shapes of the sets
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


X_train shape: (73600, 64, 64, 1)
y_train shape: (73600, 46)
X_test shape: (18400, 64, 64, 1)
y_test shape: (18400, 46)


In [8]:
# Import necessary libraries
from keras.applications import VGG19
from keras.models import Sequential
from keras.layers import Dense, Flatten
# Load the pre-trained VGG16 model without the top (fully connected) layers
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

# Freeze the layers from the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Create a new model and add the VGG16 base followed by custom dense layers
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(46, activation='softmax'))  
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

80134624/80134624 [==============================] - 29s 0us/step


In [9]:
X_train_rgb = np.repeat(X_train, 3, -1)
X_test_rgb = np.repeat(X_test, 3, -1)
# Train the model
history = model.fit(X_train_rgb, y_train, epochs=10, batch_size=32, validation_data=(X_test_rgb, y_test))

Epoch 1/10
2300/2300 [==============================] - 988s 428ms/step - loss: 0.6756 - accuracy: 0.8257 - val_loss: 0.2816 - val_accuracy: 0.9215
Epoch 2/10
2300/2300 [==============================] - 946s 411ms/step - loss: 0.2160 - accuracy: 0.9372 - val_loss: 0.1989 - val_accuracy: 0.9412
Epoch 3/10
2300/2300 [==============================] - 743s 323ms/step - loss: 0.1558 - accuracy: 0.9530 - val_loss: 0.1619 - val_accuracy: 0.9485
Epoch 4/10
2300/2300 [==============================] - 683s 297ms/step - loss: 0.1237 - accuracy: 0.9620 - val_loss: 0.1384 - val_accuracy: 0.9588
Epoch 5/10
2300/2300 [==============================] - 668s 291ms/step - loss: 0.1029 - accuracy: 0.9682 - val_loss: 0.1443 - val_accuracy: 0.9569
Epoch 6/10
2300/2300 [==============================] - 672s 292ms/step - loss: 0.0890 - accuracy: 0.9719 - val_loss: 0.1405 - val_accuracy: 0.9562
Epoch 7/10
2300/2300 [==============================] - 641s 279ms/step - loss: 0.0794 - accuracy: 0.9749 - val_